In [1]:
!pip install -q zstandard  datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00


In [ ]:
# from datasets import load_dataset

# # This takes a few minutes to run, so go grab a tea or coffee while you wait :)
# data_files = "https://mystic.the-eye.eu/public/AI/pile_preliminary_components/PUBMED_title_abstracts_2019_baseline.jsonl.zst"
# pubmed_dataset = load_dataset("json", data_files=data_files, split="train")
# pubmed_dataset

In [2]:
from datasets import load_dataset
pubmed_dataset = load_dataset("hwang2006/PUBMED_title_abstracts_2020_baseline", split="train")
pubmed_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/49 [00:00<?, ?it/s]

Dataset({
    features: ['meta', 'text'],
    num_rows: 17722096
})

In [3]:
pubmed_dataset[0]

{'meta': {'pmid': 1673585, 'language': 'eng'},
 'text': 'Cardiac beta-adrenoceptor regulation and the effects of partial agonism.\nThe in vivo effects of xamoterol on the regulation of rat cardiac beta adrenoceptors were investigated. Rats were implanted subcutaneously with osmotic minipumps and exposed to the following treatment regimens: (1) subcutaneous infusion of saline (control), isoprenaline or xamoterol for 6 days, (2) subcutaneous infusion of isoprenaline with co-administration of xamoterol for various periods up to 96 hours, and (3) subcutaneous infusion of xamoterol for up to 96 hours after previous treatment with isoprenaline for 72 hours. Xamoterol did not induce beta-adrenoceptor down-regulation after short-term (72-hour) or long-term (6-day) infusions. When coadministered with isoprenaline xamoterol did not affect the rate or extent of down-regulation induced by isoprenaline alone. In addition, recovery of beta adrenoceptors down-regulated by isoprenaline treatment was n

In [4]:
!pip install psutil

In [5]:
import psutil

# Process.memory_info is expressed in bytes, so convert to megabytes
print(f"RAM used: {psutil.Process().memory_info().rss / (1024 * 1024):.2f} MB")

RAM used: 809.50 MB


In [6]:
print(f"Number of files in dataset : {pubmed_dataset.dataset_size}")
size_gb = pubmed_dataset.dataset_size / (1024**3)
print(f"Dataset size (cache file) : {size_gb:.2f} GB")

Number of files in dataset : 24453015916
Dataset size (cache file) : 22.77 GB


In [7]:
import timeit

code_snippet = """batch_size = 1000

for idx in range(0, len(pubmed_dataset), batch_size):
    _ = pubmed_dataset[idx:idx + batch_size]
"""

time = timeit.timeit(stmt=code_snippet, number=1, globals=globals())
print(
    f"Iterated over {len(pubmed_dataset)} examples (about {size_gb:.1f} GB) in "
    f"{time:.1f}s, i.e. {size_gb/time:.3f} GB/s"
)

Iterated over 17722096 examples (about 22.8 GB) in 232.4s, i.e. 0.098 GB/s


In [ ]:
pubmed_dataset_streamed = load_dataset(
    "json", data_files="hwang2006/PUBMED_title_abstracts_2020_baseline", split="train", streaming=True
)

In [ ]:
next(iter(pubmed_dataset_streamed))

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_dataset = pubmed_dataset_streamed.map(lambda x: tokenizer(x["text"]))
next(iter(tokenized_dataset))

In [ ]:
shuffled_dataset = pubmed_dataset_streamed.shuffle(buffer_size=10_000, seed=42)
next(iter(shuffled_dataset))

In [ ]:
dataset_head = pubmed_dataset_streamed.take(5)
list(dataset_head)

In [ ]:
# Skip the first 1,000 examples and include the rest in the training set
train_dataset = shuffled_dataset.skip(1000)
# Take the first 1,000 examples for the validation set
validation_dataset = shuffled_dataset.take(1000)